# UPML
## Unification & Pattern Matching Language

(c) Dmitry Ponyatov <<dponyatov@gmail.com>> MIT 2019 All rights reserved

github: https://bitbucket.org/ponyatov/UPML

In [1]:
# %pip install -U pip
# %pip install -U -r requirements.txt

In [2]:
import os,sys
import graphviz

In [3]:
class Frame:

    def __init__(self,V):
        self.type = self.__class__.__name__.lower()
        self.val  = V
        self.slot = {}
        self.nest = []

In [4]:
hello = Frame('hello') ; print(hello)

In [5]:
class Frame(Frame):

    def __repr__(self): return self.dump()

    # full tree dump
    def dump(self,depth=0,prefix=''):
        # head
        tree = self._pad(depth) + self.head(prefix)
        # infinite recursion block
        if not depth: Frame._dump = []
        if self in Frame._dump: return tree + ' _/'
        else: Frame._dump.append(self)
        # slot{}s
        for i in self.slot:
            tree += self.slot[i].dump(depth+1,'%s = '%i)
        # nest[]ed
        idx = 0
        for j in self.nest:
            tree += j.dump(depth+1,'%i: '%idx) ; idx += 1
        # subtree dump
        return tree

    # <T:V> header only dump
    def head(self,prefix=''):
        return '%s<%s:%s> id:%x' % (prefix,self.type,self._val(),id(self))
    
    def _pad(self,depth):
        return '\n' + ' '*4 * depth
    def _val(self):
        return '%s' % self.val

In [6]:
hello = Frame('hello') ; print(hello)


<frame:hello> id:7fc298346978
